In [10]:
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [11]:
def create_image_paths(images_ids):
    images_paths = []
    img = plt.imread('C:/Users/mahmo/Desktop/Leaf-Classification/input/images/1.jpg')
    source_folder = 'C:/Users/mahmo/Desktop/Leaf-Classification/input/images/'
    for id in images_ids:
        img_path = source_folder + str(id) + '.jpg'
        images_paths.append(img_path)
    return images_paths

In [12]:
def draw_some_images(images_paths):
    fig , axis = plt.subplots(2,4)
    fig.set_figheight(10)
    fig.set_figwidth(20)
    # load images
    images = []
    for path in images_paths:
        images.append(plt.imread(path))
    i , j = 0 , 0
    for img in images:
        axis[i,j].imshow(img,cmap='gray')
        j = j + 1
        if j == 4:
            i = i + 1
            j = 0

In [13]:
def read_data(file_name):
    # read data from csv file in input folder
    cur_dir = os.getcwd()
    file_path = 'C:/Users/mahmo/Desktop/Leaf-Classification/input/' + file_name
    data = pd.read_csv(os.path.join(cur_dir,file_path))
    return data

In [15]:
def clean_data(train_data, test_data):
    X_train = train_data.drop(columns=['species','id'])
    y_train = train_data[['species']]
    # rename test variable and drop id column from the test dat
    X_test = test_data.drop(columns=['id'])
    # view correlations between columns
    correlation_matrix = X_train.corr()
    # take absolute values
    correlation_matrix_abs = correlation_matrix.abs()
    upper_matrix = correlation_matrix_abs.where(np.triu(np.ones(correlation_matrix_abs.shape),k=1).astype(np.bool_))
    # check to see if there is a column has a correlation higher than 0.9
    correlation_threshold = 0.9
    col_with_high_correlation = [col for col in upper_matrix.columns if np.any(upper_matrix[col] > correlation_threshold)]
    # drop shape columns from X_train and also X_test
    X_train = X_train.drop(columns=col_with_high_correlation)
    X_test = X_test.drop(columns=col_with_high_correlation)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    # apply the transformation to the X_test data
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    y_train_encoded = pd.get_dummies(y_train['species'])
    return X_train_scaled, y_train_encoded, X_test_scaled